In [88]:
import pandas as pd
import numpy as np
import re
import io
import warnings
import generador
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
warnings.filterwarnings('ignore')


In [89]:
paises=pd.read_excel('../data/paisess.xlsx')
pasar_nombres=pd.read_excel('../data/paises.xlsx')

#Transvulcania17

In [90]:
vulca17=pd.read_excel('../data/transvulcania/transvulcania_2017.xlsx')

In [91]:
vulca17.drop(['Dorsal', 'Unnamed: 4'], axis='columns', inplace=True)   #Damos de baja las columnas que no queremos


#Limipiamos la columna de puestos Categ donde se mezclan dos datos el puesto y la categoria y lo dejamos solo con la categoria
puestos=[]
for i in vulca17['Puesto Categ.']:
    puesto=re.findall('[a-zA-Z]+',i)
    
    if puesto:
        puestos.append("".join(puesto))
vulca17['Puesto Categ.']=puestos


#Los datos de tiempo vienen mal  asi que los limpiamos y dejamos y dejamos solo datos de tiempo
tiempos=[]
for i in range(len(vulca17)):
    vulca17['Tiempo'][i]=str(vulca17['Tiempo'][i])[:8]
    vulca17['Diff.'][i]=str(vulca17['Diff.'][i])[:8]
    
    
columns=['posicion', 'nombre','pais','pos_sex','cat','club','tiempo','media','dif']
vulca17.set_axis(columns,axis='columns',inplace=True)       #Modificamos el nombre de las columnas

vulca17=vulca17.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','cat','pos_sex','media'])     #Reordenamos las columnas

vulca17.drop(['cat'], axis='columns', inplace=True)         #Eliminamos la columna ya que no nos da ningunda info
    

In [92]:
nombres=[]
for i in range(len(vulca17.nombre)):
    nombre=str(vulca17.nombre[i]).split()
    nombre[0]=nombre[0].lower().capitalize()
    nombres.append(" ".join(nombre))
    
    
vulca17.nombre=nombres
    

In [93]:
#Cumplimentamos los nulos
vulca17.info()
vulca17.club.fillna(value='N/D', inplace=True)
vulca17.pais.fillna(value='N/D', inplace=True)
vulca17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1487 entries, 0 to 1486
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   posicion  1487 non-null   int64 
 1   tiempo    1487 non-null   object
 2   dif       1487 non-null   object
 3   nombre    1487 non-null   object
 4   club      1465 non-null   object
 5   pais      1480 non-null   object
 6   pos_sex   1487 non-null   object
 7   media     1487 non-null   object
dtypes: int64(1), object(7)
memory usage: 93.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1487 entries, 0 to 1486
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   posicion  1487 non-null   int64 
 1   tiempo    1487 non-null   object
 2   dif       1487 non-null   object
 3   nombre    1487 non-null   object
 4   club      1487 non-null   object
 5   pais      1487 non-null   object
 6   pos_sex   1487 non-null   object
 7   media     

In [94]:
#Normalizamos el nombre de los países con el resto
#generador.paises(vulca17.pais,paises,'largo','pais')

for i in range(len(vulca17.pais)):
    for j in paises['largo']:
        if str(j) in vulca17.pais[i]:
            vulca17.pais[i]= list(paises.pais[paises['largo']==j])[0]

In [95]:
#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

vulca17.drop(['pos_sex','media'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes
vulca17['sex']=generador.generador(len(vulca17))
vulca17['cat']=generador.generador(len(vulca17))
vulca17['carrera']=generador.carrera(len(vulca17),'transvulcania')
vulca17['anio']=generador.anio(len(vulca17),'2017')

vulca17=vulca17.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos

In [96]:
vulca17.tail()

,posicion,tiempo,dif,nombre,club,pais,sex,cat,carrera,anio
1482,1483,16:34:26,09:32:22,Loren HARNETT,GOTRI ADVENTURE,Irlanda,N/D,N/D,transvulcania,2017
1483,1484,16:36:45,09:34:43,Jorge VELASCO,RUNNING,España,N/D,N/D,transvulcania,2017
1484,1485,16:37:15,09:35:12,Antonio VICENTE DIAZ RODRIGUEZ,ARISTA,España,N/D,N/D,transvulcania,2017
1485,1486,16:38:44,09:36:41,Marcus RÖMER,TEAM TRICK-REISEN,Alemania,N/D,N/D,transvulcania,2017
1486,1487,16:47:04,09:45:01,Julian ALBERTO AHUMADA BETIN,PIMPOPS - TEAM,España,N/D,N/D,transvulcania,2017


In [97]:
vulca17.to_csv('../data/transvulcania/transvulcania_2017mod.xlsx', index=False)  #Cargamos el csv

#Transvulcania18

In [98]:
out_file = io.StringIO()

with open('../data/transvulcania/transvulcania2018.csv', encoding='utf8') as file:
    out_file.write(file.read().replace('""', '|').replace('"', ""))

out_file.seek(0)

vulca18 = pd.read_csv(out_file, on_bad_lines='skip',delimiter = ',', quotechar='|')

In [99]:
vulca18.drop(['link_link__96ppl href'],axis='columns', inplace=True)  #Eliminamos columnas

columns=['posicion','tiempo','nombre','pais','sex','rango_edad']
vulca18.set_axis(columns,axis='columns', inplace=True)  #Modificamos nombres de columnas

In [100]:
vulca18.rango_edad.unique()

array(['35-39', '20-34', '40-44', '45-49', '50-54', '55-59', '65-69',
       '60-64'], dtype=object)

In [101]:
#creamos una columna de categoria en base a los rangos de edad y la colocamos. 
#Aprovechamos y normalizamos el sexo tb

vulca18['cat']=vulca18['rango_edad']

for i in range(len(vulca18.rango_edad)):
    if vulca18.rango_edad[i]=='20-34' or vulca18.rango_edad[i]=='35-39':
        vulca18.cat[i]='SENIOR'
    elif vulca18.rango_edad[i]=='40-44' or vulca18.rango_edad[i]=='45-49':
        vulca18.cat[i]='VET-A'
    elif vulca18.rango_edad[i]=='50-54' or vulca18.rango_edad[i]=='55-59':
        vulca18.cat[i]='VET-B'
    else:
        vulca18.cat[i]='VET-C'
        
        
    if vulca18.sex[i]=='women':  #nornalizamos el sexo con m
        vulca18.sex[i]='F'
    elif vulca18.sex[i]=='men':  
        vulca18.sex[i]='M'
        
        
        
vulca18=vulca18.reindex(columns=['posicion','tiempo','nombre','pais','sex','cat','rango_edad'])

vulca18.cat.unique()

array(['SENIOR', 'VET-A', 'VET-B', 'VET-C'], dtype=object)

In [102]:
vulca18.columns

Index(['posicion', 'tiempo', 'nombre', 'pais', 'sex', 'cat', 'rango_edad'], dtype='object')

In [103]:
#generador.paises(vulca18.pais,pasar_nombres,'ingles','espagnol')

for i in range(len(vulca18.pais)):
    for j in pasar_nombres['ingles']:
        if str(j) in vulca18.pais[i]:
            vulca18.pais[i]= list(pasar_nombres.espagnol[pasar_nombres['ingles']==j])[0]

In [104]:
#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

vulca18.drop(['rango_edad'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes
vulca18['dif']=generador.dif(len(vulca18))
vulca18['club']=generador.generador(len(vulca18))
vulca18['carrera']=generador.carrera(len(vulca18),'transvulcania')
vulca18['anio']=generador.anio(len(vulca18),'2018')

vulca18=vulca18.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos

In [105]:
vulca18.to_csv('../data/transvulcania/transvulcania_2018mod.xlsx', index=False) #Cargamos el csv

#Transvulcania19

In [106]:
out_file = io.StringIO()

with open('../data/transvulcania/transvulcania2019.csv', encoding='utf8') as file:
    out_file.write(file.read().replace('""', '|').replace('"', ""))

out_file.seek(0)

vulca19 = pd.read_csv(out_file, on_bad_lines='skip',delimiter = ',', quotechar='|')

In [107]:
vulca19.drop(['tablescraper-selected-row href','tablescraper-selected-row src','tablescraper-selected-row src 2'], axis='columns',inplace=True)

columns=['posicion','nombre','tiempo','rango_edad','sex','pais']
vulca19.set_axis(columns,axis='columns',inplace=True)

vulca19=vulca19.reindex(columns=['posicion','tiempo','nombre','pais','sex','rango_edad'])

In [108]:
vulca19.rango_edad.unique()

array([29, 28, 25, 30, 35, 36, 39, 37, 40, 38, 27, 41, 31, 21, 33, 23, 45,
       24, 34, 32, 47, 43, 53, 49, 42, 46, 26, 50, 51, 44, 57, 52, 55, 48,
       62, 54, 69, 56, 22, 58, 63, 61, 59, 19, 60, 64], dtype=int64)

In [109]:
#creamos una columna de categoria en base a los rangos de edad y la colocamos. Tambien modificamos el sexo
vulca19['cat']=vulca19['rango_edad']

for i in range(len(vulca19.rango_edad)):
    if int(vulca19.rango_edad[i])>=20 and int(vulca19.rango_edad[i])<40:
        vulca19.cat[i]='SENIOR'
    elif int(vulca19.rango_edad[i])>=40 and int(vulca19.rango_edad[i])<50:
        vulca19.cat[i]='VET-A'
    elif int(vulca19.rango_edad[i])>=50 and int(vulca19.rango_edad[i])<60:
        vulca19.cat[i]='VET-B'
    else:
        vulca19.cat[i]='VET-C'
        
    if vulca19.sex[i]=='H':  #nornalizamos el sexo con m
        vulca19.sex[i]='M'
        
vulca19=vulca19.reindex(columns=['posicion','tiempo','nombre','pais','sex','cat','rango_edad'])

vulca19.cat.unique()

array(['SENIOR', 'VET-A', 'VET-B', 'VET-C'], dtype=object)

In [110]:

#Normalizamos el nombre de los países con el resto
for i in range(len(vulca19.pais)):
    for j in paises['largo']:
        if str(j) in vulca19.pais[i]:
            vulca19.pais[i]= list(paises.pais[paises['largo']==j])[0]

#Modificamos este valor que no se modifica al pasar la función
vulca19.pais[vulca19.pais=='ES']='España'

In [111]:
#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

vulca19.drop(['rango_edad'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes
vulca19['dif']=generador.dif(len(vulca19))
vulca19['club']=generador.generador(len(vulca19))
vulca19['carrera']=generador.carrera(len(vulca19),'transvulcania')
vulca19['anio']=generador.anio(len(vulca19),'2019')

vulca19=vulca19.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos

In [112]:
vulca19.to_csv('../data/transvulcania/transvulcania_2019mod.xlsx', index=False) #Cargamos el csv

In [113]:
vulca17.tail()

,posicion,tiempo,dif,nombre,club,pais,sex,cat,carrera,anio
1482,1483,16:34:26,09:32:22,Loren HARNETT,GOTRI ADVENTURE,Irlanda,N/D,N/D,transvulcania,2017
1483,1484,16:36:45,09:34:43,Jorge VELASCO,RUNNING,España,N/D,N/D,transvulcania,2017
1484,1485,16:37:15,09:35:12,Antonio VICENTE DIAZ RODRIGUEZ,ARISTA,España,N/D,N/D,transvulcania,2017
1485,1486,16:38:44,09:36:41,Marcus RÖMER,TEAM TRICK-REISEN,Alemania,N/D,N/D,transvulcania,2017
1486,1487,16:47:04,09:45:01,Julian ALBERTO AHUMADA BETIN,PIMPOPS - TEAM,España,N/D,N/D,transvulcania,2017


#Transvulcania22

In [114]:
out_file = io.StringIO()

with open('../data/transvulcania/transvulcania2022.csv', encoding='utf8') as file:
    out_file.write(file.read().replace('""', '|').replace('"', ""))

out_file.seek(0)

vulca22 = pd.read_csv(out_file, on_bad_lines='skip',delimiter = ',', quotechar='|')

In [115]:
vulca22.drop(['rank-sup_rankSup__TNqsa','race-progressive-ranking_linkUnderline__9ReuG href','font-d-16 2'],axis='columns',inplace=True)        #Eliminamos columnas

columns=['posicion','nombre','pais','rango_edad','club','pto_cat','pos_sex','tiempo','dif']
vulca22.set_axis(columns,axis='columns',inplace=True)       #Modificamos nombres columnas

#Las columnas posicion,pto_cat y pos_sex vienen en formato float y lo dejamos como str y con un número que tiene más sentido
for i in range(len(vulca22)):
    npos=str(vulca22.posicion[i]).find('.')
    ncat=str(vulca22.pto_cat[i]).find('.')
    nsex=str(vulca22.pos_sex[i]).find('.')
    pos=str(vulca22.posicion[i])[:npos]
    pos_cat=str(vulca22.pto_cat[i])[:ncat]
    pos_sex=str(vulca22.pos_sex[i])[:nsex]
    
    posi=re.findall('\d+',pos)
    cati=re.findall('\d+',pos_cat)
    sexi=re.findall('\d',pos_sex)
    vulca22.posicion[i]="".join(posi)
    vulca22.pto_cat[i]="".join(cati)
    vulca22.pos_sex[i]="".join(sexi)





#La columnas rangos_edad viene con información de sexo y lo podemos en una columna a parte
vulca22['sex']=vulca22['posicion']      #Creamos columna sex para meter valores
for i in range(len(vulca22.rango_edad)):
    vulca22['sex'][i]=str(vulca22.rango_edad[i])[-1]
    vulca22.rango_edad[i]=str(vulca22.rango_edad[i])[:5]
    


In [116]:
vulca22.rango_edad.unique()

array(['20-34', '45-49', '35-39', '40-44', '50-54', '55-59', 'nan',
       '60-64', '65-69', '70-74'], dtype=object)

In [117]:
#creamos una columna de categoria en base a los rangos de edad y la colocamos
vulca22['cat']=vulca22['rango_edad']

for i in range(len(vulca22.rango_edad)):
    
    vulca22.club[i]=str(vulca22.club[i]).upper()
    
    if vulca22.rango_edad[i]=='20-34' or vulca22.rango_edad[i]=='35-39':
        vulca22.cat[i]='SENIOR'
    elif vulca22.rango_edad[i]=='40-44' or vulca22.rango_edad[i]=='45-49':
        vulca22.cat[i]='VET-A'
    elif vulca22.rango_edad[i]=='50-54' or vulca22.rango_edad[i]=='55-59':
        vulca22.cat[i]='VET-B'
        
    else:
        vulca22.cat[i]='VET-C'
        


vulca22.cat.unique()

array(['SENIOR', 'VET-A', 'VET-B', 'VET-C'], dtype=object)

In [118]:
vulca22=vulca22.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','pos_sex','cat','pto_cat','rango_edad'])


In [119]:

vulca22.pais.fillna(value='N/D')
vulca22.dropna(inplace=True)


vulca22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 869 entries, 0 to 869
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   posicion    869 non-null    object
 1   tiempo      869 non-null    object
 2   dif         869 non-null    object
 3   nombre      869 non-null    object
 4   club        869 non-null    object
 5   pais        869 non-null    object
 6   sex         869 non-null    object
 7   pos_sex     869 non-null    object
 8   cat         869 non-null    object
 9   pto_cat     869 non-null    object
 10  rango_edad  869 non-null    object
dtypes: object(11)
memory usage: 81.5+ KB


In [120]:

#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

vulca22.drop(['pto_cat','pos_sex', 'rango_edad'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes

vulca22['carrera']=generador.carrera(len(vulca22),'transvulcania')
vulca22['anio']=generador.anio(len(vulca22),'2022')

vulca22=vulca22.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos



In [121]:
vulca22.reset_index(drop=True,inplace=True)

In [122]:
#Normalizamos el nombre de los países con el resto
for i in range(len(vulca22.pais)):
    for j in paises['largo']:
        if str(j) in vulca22.pais[i]:
            vulca22.pais[i]= list(paises.pais[paises['largo']==j])[0]

In [123]:
vulca22['carrera']=generador.carrera(len(vulca22),'transvulcania')
vulca22['anio']=generador.anio(len(vulca22),'2022')

In [124]:
vulca22.to_csv('../data/transvulcania/transvulcania_2022mod.xlsx', index=False) #Cargamos el csv

In [125]:
transvulcania=pd.concat([vulca17,vulca18,vulca19,vulca22])

In [126]:
transvulcania.to_csv('../data/transvulcania/transvulcania.csv',index=False, index_label=False)

In [127]:
transvulcania

,posicion,tiempo,dif,nombre,club,pais,sex,cat,carrera,anio
0,1,07:02:02,00:00:00,Timothy LEE FRERIKS,N/D,Estados Unidos,N/D,N/D,transvulcania,2017
1,2,07:18:14,00:16:12,Ludovic POMMERET,TEAM HOKA,Francia,N/D,N/D,transvulcania,2017
2,3,07:26:30,00:24:28,Zaid AIT MALEK,BUFF PRO TEAM,España,N/D,N/D,transvulcania,2017
3,4,07:28:08,00:26:04,Nicolas MARTIN,HOKA TEAM TRAIL,Francia,N/D,N/D,transvulcania,2017
4,5,07:35:04,00:33:02,Daniel JUNG,GORE RUNNING,Italia,N/D,N/D,transvulcania,2017
5,6,07:36:12,00:34:08,Hector HAINES,SCOTT RUNNING,Suecia,N/D,N/D,transvulcania,2017
6,7,07:39:06,00:37:04,Xavier THEVENARD,ASICS,Francia,N/D,N/D,transvulcania,2017
7,8,07:40:28,00:38:26,Dmitry MITYAEV,ADIDAS TERREX,Rusia,N/D,N/D,transvulcania,2017
8,9,07:40:51,00:38:47,Jason SCHLARB,"ALTRA,",Estados Unidos,N/D,N/D,transvulcania,2017
9,10,07:46:29,00:44:25,Manuel ANGUITA BAYO,ASICS-,España,N/D,N/D,transvulcania,2017
